### Fast Liquid Genaration by Packmol
[LiquidGenerator](https://github.com/matlantis-pfcc/matlantis-contrib/tree/main/matlantis_contrib_examples/liquid_generator) was provided to create the initial liquid structure.
FastLiquidGenerator uses Packmol as its engine, making it possible to generate liquid structures more than 10 times faster, even for large models.

### Install packmol
Clone the source code from git and build packmol.

In [36]:
!git clone https://github.com/m3g/packmol.git
!cd packmol && ./configure && make

Cloning into 'packmol'...
remote: Enumerating objects: 1932, done.
remote: Counting objects: 100% (479/479), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 1932 (delta 249), reused 399 (delta 221), pack-reused 1453
Receiving objects: 100% (1932/1932), 10.84 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (1069/1069), done.
Setting compiler to /usr/bin/gfortran
 ------------------------------------------------------ 
 Compiling packmol with /usr/bin/gfortran 
 Flags: -O3 --fast-math -march=native -funroll-loops 
 ------------------------------------------------------ 
 ------------------------------------------------------ 
 Packmol succesfully built.
 ------------------------------------------------------ 


### `FastLiquidGenerator` class
Although packmol can create and run its own scripts and xyz files, the `FastliquidGenerator` class can be used to create ASE Atoms from multiple SMILES.

In [37]:
import os
import subprocess as sub
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, List, Literal, Optional

import numpy as np
from ase import Atoms
from ase.io import read, write
from ase.units import _Nav
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Descriptors import ExactMolWt
from typing_extensions import TypedDict


class Composition(TypedDict):
    name: str
    smiles: str
    number: int


@dataclass
class FastLiquidGenerator:
    outputname: str = "packmol.xyz"
    outputdir: str = "output"
    composition: Dict[str, Composition] = field(default_factory=dict)
    density: float = 0.5  # units:g/cm^3
    pbc: Optional[List[bool]] = field(default_factory=lambda: [True, True, True])
    packmol_bin: Optional[Path] = "packmol/packmol"
    inputname: Optional[str] = "packmol.inp"
    filetype: Optional[
        str
    ] = "xyz"  # same as filetype in packmol. Please see https://m3g.github.io/packmol/userguide.shtml
    tolerance: Optional[
        float
    ] = 2.0  # same as tolerance in packmol. Please see https://m3g.github.io/packmol/userguide.shtml
    margin: Optional[float] = 1.0  # margin for box and atom
    cell: Optional[Dict[Literal["lx", "ly", "lz"], float]] = field(default_factory=dict)
    verbose: Optional[bool] = False

    def __post_init__(self) -> None:
        if not os.path.isdir(self.outputdir):
            os.makedirs(self.outputdir)

        if not self.cell:
            total_mass = 0
            for name, props in self.composition.items():
                mol = Chem.MolFromSmiles(props["smiles"])
                mol = AllChem.AddHs(mol)
                total_mass += props["number"] * ExactMolWt(mol)
            total_mass /= _Nav
            l = round((total_mass / self.density) ** (1 / 3) * 10**8, 2)
            self.cell = {"lx": l, "ly": l, "lz": l}

    def _smiles_to_atoms(self, smiles: str) -> Atoms:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        atoms = Atoms(
            positions=mol.GetConformer().GetPositions(),
            numbers=np.array([a.GetAtomicNum() for a in mol.GetAtoms()]),
        )
        return atoms

    def run(self) -> Atoms:
        with open(os.path.join(self.outputdir, self.inputname), "w") as f:
            f.write(f"tolerance {self.tolerance}\n")
            f.write(f"filetype {self.filetype}\n")
            f.write(f"output {os.path.join(self.outputdir, self.outputname)}\n")
            f.write(f"\n")
            for name, props in self.composition.items():
                structure_path = os.path.join(self.outputdir, name + "." + self.filetype)
                f.write(f"structure {structure_path}\n")
                f.write(f"  number {props['number']}\n")
                f.write(
                    f"  inside box {self.margin} {self.margin} {self.margin} {self.cell['lx']-self.margin} {self.cell['ly']-self.margin} {self.cell['lz']-self.margin}\n"
                )
                f.write(f"end structure\n")
                atoms = self._smiles_to_atoms(props["smiles"])
                write(structure_path, atoms)

        cmd = [self.packmol_bin]
        stdout = None if self.verbose else sub.DEVNULL

        with open(os.path.join(self.outputdir, self.inputname), "rt") as f:
            sub.run(cmd, text=True, stdout=stdout, stdin=f)

        packed_atoms = read(os.path.join(self.outputdir, self.outputname))
        packed_atoms.set_cell([self.cell["lx"], self.cell["ly"], self.cell["lz"]])
        packed_atoms.set_pbc(self.pbc)

        write(os.path.join(self.outputdir, self.outputname), packed_atoms)

        return packed_atoms

### Example: Mixture of 1000 water moleculs and 1000 ethanol molecules

In [38]:
liq_generator = FastLiquidGenerator(
    density=1.0,
    composition={
        "water": {"smiles": "O", "number": 1000},
        "ehtanol": {"smiles": "CCO", "number": 1000},
    },
    outputname="mixture.xyz",
    filetype="xyz",
    outputdir="output"
)
atoms = liq_generator.run()

In [39]:
atoms

Atoms(symbols='C2000H8000O2000', pbc=True, cell=[47.38, 47.38, 47.38])